## **Load libraries and data**

In [43]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

Requirement already up-to-date: pythainlp in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [44]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [22]:
df = pd.read_csv('Wongnai Reviews.csv')

In [23]:
df.tail()

,Review ID,Review,Rating
39995,39996,รู้จักร้านนี้จากวงใน ร้านอยู่ในอาคารพาณิชย์ตรง...,4
39996,39997,ร้านซูชิอาหารญี่ปุ่น อยู่ตรงสะพานลอย เกษตรประต...,4
39997,39998,"""Cantina Wine Bar & Italian Kitchen"" ร้านเล็กๆ...",5
39998,39999,ร้านเค้กน่ารักๆ ตรงชั้นล่างของห้างเซ็นทรัลลาดพ...,3
39999,40000,วันนี้มากินกันไกลถึงแม่สอดจังหวัดตากติดกับชายแ...,3


Tokenize Words

In [28]:
def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    merged = merged + ',' + word
  return merged[1:]

In [29]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [30]:
df.tail()

,Review ID,Review,Rating,Review_tokenized
39995,39996,รู้จักร้านนี้จากวงใน ร้านอยู่ในอาคารพาณิชย์ตรง...,4,"รู้จัก,ร้าน,นี้,จาก,วงใน, ,ร้าน,อยู่,ใน,อาคาร,..."
39996,39997,ร้านซูชิอาหารญี่ปุ่น อยู่ตรงสะพานลอย เกษตรประต...,4,"ร้าน,ซูชิ,อาหาร,ญี่ปุ่น, ,อยู่,ตรง,สะพานลอย, ,..."
39997,39998,"""Cantina Wine Bar & Italian Kitchen"" ร้านเล็กๆ...",5,""",Cantina, ,Wine, ,Bar, ,&, ,Italian, ,Kitchen..."
39998,39999,ร้านเค้กน่ารักๆ ตรงชั้นล่างของห้างเซ็นทรัลลาดพ...,3,"ร้าน,เค้ก,น่ารัก,ๆ, ,ตรง,ชั้นล่าง,ของ,ห้าง,เซ็..."
39999,40000,วันนี้มากินกันไกลถึงแม่สอดจังหวัดตากติดกับชายแ...,3,"วันนี้,มา,กินกัน,ไกล,ถึง,แม่,สอด,จังหวัด,ตาก,ต..."


## **Create Dictionary**

In [33]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [34]:
print(dictionary.token2id.keys())

dict_keys(['\n', ' ', ')', '150', '2', 'กก', 'กรอบ', 'กาแฟ', 'กิน', 'ก็', 'ก่อน', 'ขนมหวาน', 'ขนมไทย', 'คง', 'คาว', 'ชั้น', 'ซึ่ง', 'ซ้ำ', 'ดี', 'ต้อง', 'ทอดมัน', 'ทาน', 'นะ', 'น้ำผึ้ง', 'บาท', 'ประทับใจ', 'พะแนง', 'ฟรี', 'มัน', 'มา', 'มาก', 'มี', 'รับ', 'ราคา', 'ราด', 'ร้านอาหาร', 'ลาบ', 'ว', 'สั่ง', 'หลัก', 'หัวปลี', 'ห่อ', 'ห้อง', 'ห้องน้ำ', 'อยู่', 'อย่า', 'อร่อย', 'อะ', 'เกิน', 'เกินไป', 'เข้ามา', 'เจอ', 'เท่าไหร่', 'เบิ้ล', 'เลย', 'เลิก', 'เลี้ยว', 'เล็กน้อย', 'เวอร์', 'เห้ย', 'เออ', 'เอา', 'แค่', 'แต่', 'แปลก', 'แพง', 'แล้ว', 'ใหญ่', 'ให้', 'ไข่', 'ไข่ต้ม', 'ไป', 'ไม่', 'ไหว', 'ๆ', '  ', '"ฮก', '(', '-', '3', '5', '500', 'กุ๊ก', 'ของ', 'ขาย', 'ขึ้นไป', 'ข้าวผัด', 'คน', 'คล้ายๆ', 'คือ', 'ค่ะ', 'จะ', 'จาก', 'จาน', 'จีน', 'จึง', 'ชอบ', 'ชื่อดัง', 'ซิ่ว', 'ซุป', 'ซ้ายมือ', 'ตัว', 'ตั้งอยู่', 'ตาม', 'ถ.', 'ถนน', 'ถึง', 'ทาง', 'ที่', 'ที่นี่', 'นครชัยศรี', 'นอกนั้น', 'นี้', 'น้ำ', 'น้ำจิ้ม', 'น้ำพริกเผา', 'น้ำมัน', 'บอก', 'บ้าน', 'ประมาณ', 'ปลา', 'ปิ่น', 'ปูทะเล', 'ผัก', 'ผัด', 'พร้อม

In [37]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]

In [38]:
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [48]:
num_topics = 20
chunksize = 1000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


CPU times: user 32min 16s, sys: 5min 29s, total: 37min 46s
Wall time: 31min 18s


In [49]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.249078 -0.133195       1        1  36.496037
13    -0.237559 -0.115965       2        1  17.162252
18    -0.220000 -0.178032       3        1   8.431159
12    -0.161348 -0.217005       4        1   7.942654
9     -0.088516 -0.132140       5        1   5.815927
4     -0.195349  0.146322       6        1   5.797695
8     -0.097348  0.273695       7        1   3.409501
15    -0.082012 -0.005881       8        1   3.164808
6     -0.104973  0.231811       9        1   2.937327
2     -0.067270  0.280575      10        1   2.251537
5     -0.008938 -0.154719      11        1   1.691119
10    -0.027479  0.176331      12        1   1.305915
0      0.178519 -0.065571      13        1   0.683645
14     0.131783  0.131209      14        1   0.565396
16     0.180912  0.033480      15        1   0.504592
19     0.200204 -0.055712      16        1   0.415285
11     0.216449 -0.050268      17        1   0.413158
1      0.217679 -0.060264      18        1   0.392029
3      0.204840 -0.054261      19        1   0.388651
7      0.209485 -0.050411      20        1   0.231312, topic_info=             Term          Freq         Total Category  logprob  loglift
1                  1.047418e+06  1.047418e+06  Default  30.0000  30.0000
0              \n  2.075340e+05  2.075340e+05  Default  29.0000  29.0000
196          ครับ  3.112800e+04  3.112800e+04  Default  28.0000  28.0000
137          ร้าน  1.010540e+05  1.010540e+05  Default  27.0000  27.0000
75                 1.964700e+04  1.964700e+04  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
12246         บูธ  6.695446e+01  6.714592e+01  Topic20  -5.4172   6.0663
11253        เบอร  6.854483e+01  6.878106e+01  Topic20  -5.3937   6.0657
11611         ม้า  9.887135e+01  1.589599e+02  Topic20  -5.0274   5.5943
18640  เอสเพรสโซ่  7.884333e+01  1.352590e+02  Topic20  -5.2538   5.5294
2405           ขา  9.093429e+01  1.648144e+03  Topic20  -5.1111   3.1719

[1126 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
758       19  0.999966            
0          5  0.999999          \n
1          1  0.393524            
1          2  0.115745            
1          3  0.116422            
...      ...       ...         ...
25295     20  1.000386        ⭐⭐⭐⭐
32987     20  1.001540          ⭐️
25942      5  0.998543    ⭐️⭐️⭐️⭐️
26045     10  0.996492  ⭐️⭐️⭐️⭐️⭐️
41784     10  0.997477           》

[2164 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 14, 19, 13, 10, 5, 9, 16, 7, 3, 6, 11, 1, 15, 17, 20, 12, 2, 4, 8])